In [25]:
import os
from os import listdir
import pandas as pd
import io
from io import BytesIO

Pindel = 'Pindel/'
CNV = 'CNV/'

In [26]:
pindel_files = [f for f in listdir(Pindel)]
CNV_files = [f for f in listdir(CNV)]

In [27]:
len(pindel_files), len(CNV_files) #sorted the same way, no need to check if compared files are from the same individual

(64, 64)

In [28]:
def intersect(a, b):
    if len(a) > len(b): #change
        a, b = b, a

    listaC = [[0,0]]
    for i, el_a in enumerate(a):
        for j, el_b in enumerate(b):

            if el_a[0] <= el_b[0] <= el_b[1] <= el_a[1]: #inside (both b) ABBA
                dodane = False 

                for k, el_c in enumerate(listaC):
                    if el_b[0] <= el_c[0] <= el_c[1] <= el_b[1]:
                        listaC[k] = [el_b[0], el_b[1]] #entend existing record
                        dodane = True
                    elif el_c[0] <= el_b[0] <= el_b[1] <= el_c[1]:
                        dodane = True
                        pass #it already exist
                    elif el_c[0] <= el_b[0] <= el_c[1] <= el_b[1]:
                        dodane = True
                        listaC[k] = [el_c[0], el_b[1]] #entend existing record
                    elif el_b[0] <= el_c[0] <= el_b[1] <= el_c[1]:
                        dodane = True
                        listaC[k] = [el_b[0], el_c[1]] #entend existing record
                    else:
                        dodane = False
                if dodane == False:
                        listaC.append([el_b[0], el_b[1]])
                        
            elif el_a[0] <= el_b[0] <= el_a[1] <= el_b[1]: #cut from right ABAB
                dodane = False 
       
                for k, el_c in enumerate(listaC):
                    if el_b[0] <= el_c[0] <= el_c[1] <= el_a[1]:
                        dodane = True 
                        listaC[k] = [el_b[0], el_a[1]] #entend existing record
                    elif el_c[0] <= el_b[0] <= el_a[1] <= el_c[1]:
                        dodane = True 
                        pass #it already exist
                    elif el_c[0] <= el_b[0] <= el_c[1] <= el_a[1]:
                        dodane = True 
                        listaC[k] = [el_c[0], el_a[1]] #entend existing record
                    elif el_b[0] <= el_c[0] <= el_a[1] <= el_c[1]:
                        dodane = True 
                        listaC[k] = [el_b[0], el_c[1]] #entend existing record
                    else:
                        dodane = False
                if dodane == False:
                        listaC.append([el_b[0], el_a[1]])
                        
            elif el_b[0] <= el_a[0] <= el_a[1] <= el_b[1]: #both BAAB
                dodane = False 
 
                for k, el_c in enumerate(listaC):
                    if el_a[0] <= el_c[0] <= el_c[1] <= el_a[1]:
                        dodane = True 
                        listaC[k] = [el_a[0], el_a[1]] #entend existing record
                    elif el_c[0] <= el_a[0] <= el_a[1] <= el_c[1]:
                        dodane = True 
                        pass #it already exist
                    elif el_c[0] <= el_a[0] <= el_c[1] <= el_a[1]:
                        dodane = True 
                        listaC[k] = [el_c[0], el_a[1]] #entend existing record
                    elif el_a[0] <= el_c[0] <= el_a[1] <= el_c[1]:
                        dodane = True 
                        listaC[k] = [el_a[0], el_c[1]] #entend existing record
                    else:
                        dodane = False
                if dodane == False:
                        listaC.append([el_a[0], el_a[1]])

                        
            elif el_b[0] <= el_a[0] <= el_b[1] <= el_a[1]: #BABA
                dodane = False 
                
                for k, el_c in enumerate(listaC):
                    if el_a[0] <= el_c[0] <= el_c[1] <= el_b[1]:
                        listaC[k] = [el_b[0], el_a[1]] #entend existing record
                    elif el_c[0] <= el_a[0] <= el_b[1] <= el_c[1]:
                        pass #it already exist
                    elif el_c[0] <= el_a[0] <= el_c[1] <= el_b[1]:
                        listaC[k] = [el_c[0], el_a[1]] #entend existing record
                    elif el_a[0] <= el_c[0] <= el_b[1] <= el_c[1]:
                        listaC[k] = [el_a[0], el_c[1]] #entend existing record
                    else:
                        dodane = False
                if dodane == False:
                        listaC.append([el_a[0], el_b[1]])
            else:
                pass
    return listaC

In [97]:
for i in range(0,1): #0,64 by files
    file_pindel = pd.read_csv("Pindel/"+pindel_files[i], names=['Chr', "Start", "End"],  sep=" ", dtype={'Chr':'str', 'Start':'int', 'End':'int' })
    file_cnv = pd.read_csv("CNV/"+CNV_files[i], names=['Chr', "Start", "End"],  sep=" ", dtype={'Chr':'str', 'Start':'int', 'End':'int' })                            
    df_connected = pd.DataFrame(columns=['Chr', "Start", "End"])
#     print(df_connected.head())
#     print(len(file_pindel))
#     print(len(file_cnv))
    for j in range(1,3): #30 docelowo #by chromosoms Tutaj niestety chr musiało być stringiem bo jest X też
        inter1 =  file_pindel[['Start', 'End']][file_pindel.Chr==str(j)].values.tolist()
        inter2 =  file_cnv[['Start', 'End']][file_cnv.Chr==str(j)].values.tolist()
        listaC = intersect(inter1, inter2)
        print("tyle wspolnych regionow", len(listaC), "chromosom", j)

        df_new = pd.DataFrame(listaC, columns =['Start', 'End'])[1:] #tutaj usuwam moją zainicjowaną długość listy
        df_new.insert(0, "Chr", [str(j) for i in range(len(listaC[1:]))])
        df_connected = pd.concat([df_connected,df_new])
        
    #Save for this individual
    df_connected.to_csv("Connected/"+str(pindel_files[0][:-13])+ ".csv", index=False)

    

Empty DataFrame
Columns: [Chr, Start, End]
Index: []
1538733
18987
tyle wspolnych regionow 1439 chromosom 1
tyle wspolnych regionow 1098 chromosom 2


In [ ]:
#NIEUDANE 

In [ ]:
def merge_intervals(intervals):
    sort = sorted(intervals, key=lambda tuples: tuples[0])
    merged = []

    for tuples in sort:
        if not merged:
            merged.append(tuples)
        else:
            b = merged.pop()
            if b[1] >= tuples[0]:
                new_tup = tuple([b[0], tuples[1]])
                merged.append(new_tup)
            else:
                merged.append(b)
                merged.append(tuples)
    return merged


intervals1 = file_pindel[['Start', 'End']][file_pindel.Chr=='1']
intervals2 = file_cnv[['Start', 'End']][file_cnv.Chr=='1']
tuples = [tuple(x) for x in intervals1.values] + [tuple(x) for x in intervals2.values]
tuples
merge_intervals(tuples)

In [ ]:
def findIntersection(intervals,N): 
    l = intervals[0][0] 
    r = intervals[0][1] 

    for i in range(1,N): 
        if (intervals[i][0] > r or intervals[i][1] < l): 
#             print(-/1)
            pass
        else: 
            l = max(l, intervals[i][0]) 
            r = min(r, intervals[i][1])       
    print("[", l,", ",r,"]") 

intervals= [ 
            [ 1, 6 ], 
            [ 2, 8 ], 
            [ 3, 10 ], 
            [ 5, 8 ] 
            ] 
N =len(intervals) 
findIntersection(intervals, N) 

In [ ]:
a = tuples
b = []
for begin,end in sorted(a):
    if b and b[-1][1] >= begin - 1:
        b[-1] = (b[-1][0], end)
    else:
        b.append((begin, end))
        
sorted(tuples)